# DEMO: Working with the Databricks File System (DBFS)  
---
During this demo we will learn more about working with the underlying Databricks File System or DBFS.  
Understanding how manipulate the filesystem using `dbutils` will help you be a more efficient Azure Databricks User.

Run the following cell to see all the different families of `dbutils` funtions

In [3]:
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

Here we see various groups of functions, some are marked **EXPERIMENTAL**. For this demo, we will focus on the first group, the `dbutils.fs` group of functions.  
  
The `dbutils.fs` group of functions allow users to manipulate the Databricks File System or DBFS. There are various ways you can interact with DBFS, run the following cell to show all the different types of functions within `dbutils.fs`.

In [5]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

In *fsutils*, you probably notice some familar shell commands. Commands like `ls`, `mkdirs`, `rm`, and the others mirror their shell counterparts in terms of functionality.  
  
In the *mount* category you have functions that relate to the ability for Azure Databricks users to *mount* storage sinks like Azure Data Lake Store or Azure Blob Storage.  
  
**Mounting Storage** is a topic that we will cover later on in this section, for now let us focus on the *fsutils* commands.

Let's use `mkdirs` to create some empty directories.

In [8]:
dbutils.fs.mkdirs("source")
dbutils.fs.mkdirs("sink")

Out[ 4 ]: True

Let's check to make sure they were created using the `ls` command.

In [10]:
dbutils.fs.ls("/")

Out[ 11 ]: 
[FileInfo(path='dbfs:/FileStore/', name='FileStore/', size=0),
 FileInfo(path='dbfs:/data/', name='data/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0),
 FileInfo(path='dbfs:/databricks-results/', name='databricks-results/', size=0),
 FileInfo(path='dbfs:/mnt/', name='mnt/', size=0),
 FileInfo(path='dbfs:/sink/', name='sink/', size=0),
 FileInfo(path='dbfs:/source/', name='source/', size=0),
 FileInfo(path='dbfs:/tmp/', name='tmp/', size=0),
 FileInfo(path='dbfs:/workshopFiles/', name='workshopFiles/', size=0)]

If you look at the results above you can see the two directories that we made using the `mkdirs` command. 
  
Using the `put` command we can create some small files that we can interact with using the folders we just created using `mkdir`

In [12]:
dbutils.fs.put("/source/file1.txt", "Hello World!")
dbutils.fs.put("/source/file2.txt", "Hello World!")
dbutils.fs.put("/source/file3.txt", "Hello World!")
dbutils.fs.put("/source/file4.txt", "Hello World!")
dbutils.fs.put("/source/file5.txt", "Hello World!")

Wrote 12 bytes.
Wrote 12 bytes.
Wrote 12 bytes.
Wrote 12 bytes.
Wrote 12 bytes.
 Out[ 13 ]: True

Now let's take a quick look to make sure they were written to the folder.

In [14]:
dbutils.fs.ls("/source")

Out[ 14 ]: 
[FileInfo(path='dbfs:/source/file1.txt', name='file1.txt', size=12),
 FileInfo(path='dbfs:/source/file2.txt', name='file2.txt', size=12),
 FileInfo(path='dbfs:/source/file3.txt', name='file3.txt', size=12),
 FileInfo(path='dbfs:/source/file4.txt', name='file4.txt', size=12),
 FileInfo(path='dbfs:/source/file5.txt', name='file5.txt', size=12)]

Okay, so we have created 2 directories (source, sink), and we have created some files in the source folder. Now lets try moving files around DBFS using the `mv` command.

In [16]:
files = [filepath[0] for filepath in dbutils.fs.ls("/source")]
sinkPath = "/sink"
for file in files:
  dbutils.fs.mv(file, sinkPath)

Hmmmm...that command above wasn't just using `dbutils` commands! Yes, that's right. You can incorporate `dbutils` commands into regular Python code. This allows you to manipualte the DBFS in even more ways than if you just used the commands on their own. Above we use a simple list comprehension to grab the actual filepaths of the files within the Sink folder and then iterate over the variable `files` moving each file to the sink directory.  

Now to prove this Pythonic magic is real, let's use the `ls` command one more time to show the files moved into the Sink directory.

In [18]:
dbutils.fs.ls("/sink")

Out[ 24 ]: 
[FileInfo(path='dbfs:/sink/file1.txt', name='file1.txt', size=12),
 FileInfo(path='dbfs:/sink/file2.txt', name='file2.txt', size=12),
 FileInfo(path='dbfs:/sink/file3.txt', name='file3.txt', size=12),
 FileInfo(path='dbfs:/sink/file4.txt', name='file4.txt', size=12),
 FileInfo(path='dbfs:/sink/file5.txt', name='file5.txt', size=12)]